# numpy测试

In [ ]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi, 200)
y = np.sin(x)

fig, ax = plt.subplots()
ax.plot(x, y)
plt.show()

# 频谱提取

In [ ]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt

%matplotlib tk

CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

## 单次提取

In [ ]:
p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

data = stream.read(CHUNK)
# len(data)
data_int = np.array(struct.unpack(str(2 * CHUNK) +'B', data)).astype('uint8')[::2]+127
# feel to be like this...
fig, ax = plt.subplots() # 创建图形窗口

ax.plot(data_int, '-') # 添加对象
plt.show() # 显示

#first version

## 实时显示

In [ ]:
p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

fig, ax = plt.subplots()
x = np.arange(0, 2 * CHUNK, 2)
line, = ax.plot(x, np.random.rand(CHUNK))
ax.set_ylim(-128,128)
ax.set_xlim(0,CHUNK)
# 绘图窗口设置

while True:
    data = stream.read(CHUNK)
    data_int = np.array(struct.unpack(str(2 * CHUNK) +'B', data)).astype('b')[::2]
    line.set_ydata(data_int)
    fig.canvas.draw()
    fig.canvas.flush_events()

# scipy fftpack使用

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft

# 生成示例信号
fs = 1000 # 采样率
t = np.arange(0, 1, 1/fs) #时间数组
f = 5 # 信号频率
signal = np.sin(2 * np.pi * f * t) #np.sin(是专门用来计算数组的函数)

# 进行fft
fft_result = fft(signal)

# 计算频率轴
freq = np.fft.fftfreq(len(fft_result), 1/fs)

# 只取正频率部分
positive_freq = freq[:len(freq)//2]
magnitude_spectrum = np.abs(fft_result[:len(fft_result)//2])

# 绘制原始信号和频谱图
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(t, signal)
plt.title('Original Signal')

plt.subplot(2, 1, 2)
plt.plot(positive_freq, magnitude_spectrum)
plt.title('Magnitude Spectrum')

plt.tight_layout()
plt.show()

# 通过使用FFT函数得到的结果，结合np.fft.fftfreq函数提供的频率轴，
# 你可以得到一个描述信号在频域上的振幅与频率之间的关系。

## 示例

In [ ]:
x = np.arange(0, 2 * CHUNK,2)
plt.plot(np.abs(fft(np.sin(2 * x))))

# 数据实时更新

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 创建初始数据
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

# 绘制初始图形
fig, ax = plt.subplots()
line, = ax.plot(x, y)
plt.show(block=False)  # 以非阻塞方式显示图形窗口

# 模拟动态更新数据
for i in range(100):
    # 生成新数据
    new_y = np.sin(x + i * 0.1)
    
    # 更新线的 y 数据
    line.set_ydata(new_y)
    
    # 刷新图形
    fig.canvas.draw()
    fig.canvas.flush_events()


# 频谱分析

In [ ]:
# spectrum!(forier analasys?)

In [2]:
import pyaudio
import struct
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.fftpack import fft
from tkinter import TclError

%matplotlib tk

CHUNK = 1024 * 2
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

In [ ]:
# create figure and axes
fig, (ax, ax2) = plt.subplots(2, figsize=(15,8))

p = pyaudio.PyAudio()
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK,2)
x_fft = np.linspace(0, RATE, CHUNK)
# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)
line_fft, = ax2.plot(x_fft, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes

#show the plot
plt.show(block=False)# 不会等待用户关闭图形窗口，继续执行

print('stream started')

# for measuring frame rate
frame_count = 0
# start_time = time.time()

# loop
while True:
    data = stream.read(CHUNK)
    data_int = struct.unpack(str(2 * CHUNK) +'B', data)
    data_np = np.array(data_int).astype('uint8')[::2]+127
    
    line.set_ydata(data_np)

    y_fft = fft(data_int)
    line_fft.set_ydata(np.abs(y_fft[0:CHUNK]) * 2 / (256 * CHUNK))

    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except KeyboardInterrupt:
        print('Ctrl+C pressed. Exiting...')
        stream.stop_stream()
        stream.close()
        p.terminate()
        break

In [ ]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# Signal range is -32k to 32k
# limiting amplitude to +/- 4k
AMPLITUDE_LIMIT = 8096

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-AMPLITUDE_LIMIT, AMPLITUDE_LIMIT)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[-AMPLITUDE_LIMIT, 0, AMPLITUDE_LIMIT])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)    

    data_np = np.frombuffer(data, dtype='h')
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_np)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (512 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
